<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 14:12:56
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 94.38 K (0.68%)
Current PnL: -22.25 L (-14.83%)
CY Booked + Current PnL: -10.35 L (-6.9%)
-------------------
Total profit:  1.59 L
Total loss:  -23.85 L
-------------------
Total Booked + Current PnL: 16.63 L (13.5%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.55%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.44 L (64.24%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.71,213336.0,8836.0,14464.0,0.32,4.32,6.78,11.39,10.0,0.61,1.54,13.66,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,98.08,53.0,M-SC,1.51,85301.0,-15476.0,15567.0,0.89,-15.36,18.25,0.09,245.0,-0.99,0.62,12.07,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.61,67.0,X-MC,1.72,148562.0,12429.0,18689.0,8.24,9.13,12.58,22.86,91.0,0.67,1.08,22.89,X40,ATH,INSURANCE
83,VOLTAS,1530.00,0.04,55.0,X-MC,1.06,209955.0,18213.0,19547.0,0.57,9.50,9.31,19.69,99.0,0.93,1.52,16.94,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,47.0,X-LC,3.06,214640.0,4274.0,24512.0,0.07,2.03,11.42,13.68,37.0,0.17,1.55,18.89,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-13.61,67.0,X-MC,1.72,148562.0,12429.0,18689.0,8.24,9.13,12.58,22.86,91.0,0.67,1.08,22.89,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.08,62.0,X-LC,2.40,285995.0,20569.0,36236.0,3.97,7.75,12.67,21.40,11.0,0.57,2.07,14.91,XY25,NTT,FMCG
81,VALIANTORG,838.00,-283.41,31.0,H-SC,3.52,101847.0,-61758.0,183070.0,2.97,-37.75,179.75,74.15,139.0,-0.34,0.74,30.55,XR,NTT,CHEMICALS
5,ASIANPAINT,3465.66,-17.11,46.0,X-LC,5.36,209299.0,-42469.0,95671.0,2.60,-16.87,45.71,21.13,27.0,-0.44,1.52,12.58,X40,ATH,PAINTS
25,GREENPANEL,537.00,217.83,49.0,M-SC,3.90,141959.0,-41119.0,120097.0,2.29,-22.46,84.60,43.14,230.0,-0.34,1.03,33.53,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.0,-23.17,39.0,X-MC,2.12,132233.0,-3555.0,46308.0,-2.17,-2.62,35.02,31.48,107.0,-0.08,0.96,9.18,X40,ATH,INSURANCE
15,CAMPUS,393.0,-24.03,54.0,M-SC,2.69,158296.0,-9479.0,65329.0,-2.09,-5.65,41.27,33.28,210.0,-0.15,1.15,22.69,XY24,NTT,FOOTWEAR
38,INDUSINDBK,1800.0,241.74,45.0,L-MC,11.24,41929.0,-35278.0,60671.0,-1.97,-45.69,144.70,32.89,258.0,-0.58,0.30,15.47,XR,NTT,BANKS
46,LAOPALA,464.0,95.46,30.0,H-SC,1.51,89415.0,-41690.0,92938.0,-1.61,-31.80,103.94,39.09,142.0,-0.45,0.65,15.16,AR,NTT,CERAMICS
72,TATAMOTORS,1065.0,-55.15,12.0,X-LC,3.35,141352.0,-135171.0,245246.0,-1.53,-48.88,173.50,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.36,149328.0,108.0,144072.0,0.70,0.07,96.48,96.62,208.0,0.0,1.08,61.61,XY25,NTT,FINANCE
59,SAIL,228.00,42.74,48.0,M-MC,12.29,224876.0,-86.0,167735.0,1.41,-0.04,74.59,74.53,192.0,-0.0,1.63,32.20,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.27,123909.0,573.0,131059.0,0.13,0.46,105.77,106.72,119.0,0.0,0.90,23.95,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,106.53,58.0,H-SC,11.51,130140.0,-12339.0,31858.0,1.30,-8.66,24.48,13.70,163.0,-0.39,0.94,53.60,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.08,53.0,M-SC,1.51,85301.0,-15476.0,15567.0,0.89,-15.36,18.25,0.09,245.0,-0.99,0.62,12.07,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,51.0,H-SC,1.86,222255.0,-47412.0,83746.0,0.62,-17.58,37.68,13.47,138.0,-0.57,1.61,13.01,XY24,NTT,PAINTS
17,CERA,9475.0,-23.63,41.0,H-SC,2.11,139265.0,-36638.0,78657.0,-0.59,-20.83,56.48,23.89,149.0,-0.47,1.01,20.27,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.89,42.0,H-MC,7.03,101678.0,-30157.0,72385.0,0.31,-22.87,71.19,32.03,98.0,-0.42,0.74,13.89,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.30,207762.0,4592.0,37543.0,2.01,2.26,18.07,20.74,55.0,0.12,1.50,7.93,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.83,69.0,M-LC,8.91,222676.0,17600.0,94971.0,1.24,8.58,42.65,54.89,52.0,0.19,1.61,34.75,X5K,ATH,METALS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.77,154876.0,3931.0,105517.0,0.56,2.60,68.13,72.51,53.0,0.04,1.12,8.78,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,55.0,M-LC,9.51,158179.0,17339.0,27159.0,1.55,12.31,17.17,31.59,66.0,0.64,1.15,42.06,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.73,194142.0,14334.0,98081.0,1.55,7.97,50.52,62.52,88.0,0.15,1.41,44.78,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.83,69.0,M-LC,8.91,222676.0,17600.0,94971.0,1.24,8.58,42.65,54.89,52.0,0.19,1.61,34.75,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.73,194142.0,14334.0,98081.0,1.55,7.97,50.52,62.52,88.0,0.15,1.41,44.78,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.77,154876.0,3931.0,105517.0,0.56,2.60,68.13,72.51,53.0,0.04,1.12,8.78,XR,NTT,IT
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.27,123909.0,573.0,131059.0,0.13,0.46,105.77,106.72,119.0,0.00,0.90,23.95,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-47.73,38.0,M-SC,3.97,88440.0,-3058.0,81807.0,2.18,-3.34,92.50,86.07,223.0,-0.04,0.64,28.43,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.35,141352.0,-135171.0,245246.0,-1.53,-48.88,173.50,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51443.0,-13563.0,164489.0,0.32,-20.86,319.75,232.18,198.0,-0.08,0.37,0.32,XY24,NTT,MISC
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.42,93350.0,-35320.0,80617.0,0.04,-27.45,86.36,35.20,219.0,-0.44,0.68,7.14,X40,NTT,FOOTWEAR
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79872.0,-65288.0,141214.0,-0.50,-44.98,176.80,52.31,136.0,-0.46,0.58,6.91,X40N,NTT,FOOTWEAR
28,HAVELLS,2069.16,-7.74,35.0,X-MC,1.50,228649.0,-19082.0,94135.0,1.15,-7.70,41.17,30.30,92.0,-0.20,1.66,4.57,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.61,46.0,H-LC,0.93,156550.0,-29545.0,77023.0,0.76,-15.88,49.20,25.51,15.0,-0.38,1.13,15.37,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.04,55.0,X-MC,1.06,209955.0,18213.0,19547.0,0.57,9.50,9.31,19.69,99.0,0.93,1.52,16.94,XY25,NTT,AC
28,HAVELLS,2069.16,-7.74,35.0,X-MC,1.50,228649.0,-19082.0,94135.0,1.15,-7.70,41.17,30.30,92.0,-0.20,1.66,4.57,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.61,67.0,X-MC,1.72,148562.0,12429.0,18689.0,8.24,9.13,12.58,22.86,91.0,0.67,1.08,22.89,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.08,62.0,X-LC,2.40,285995.0,20569.0,36236.0,3.97,7.75,12.67,21.40,11.0,0.57,2.07,14.91,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.35,141352.0,-135171.0,245246.0,-1.53,-48.88,173.50,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51443.0,-13563.0,164489.0,0.32,-20.86,319.75,232.18,198.0,-0.08,0.37,0.32,XY24,NTT,MISC
82,VBL,671.64,-21.25,40.0,X-LC,10.67,283495.0,-32347.0,145008.0,0.62,-10.24,51.15,35.67,5.0,-0.22,2.05,2.19,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.75,36.0,X-MC,5.66,177481.0,-23025.0,120172.0,0.92,-11.48,67.71,48.45,90.0,-0.19,1.28,2.35,X40N,ATH,FINANCE
41,ITC,452.00,-39.61,43.0,X-LC,2.83,195258.0,-4880.0,25774.0,0.63,-2.44,13.20,10.44,4.0,-0.19,1.41,2.99,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51443.0,-13563.0,164489.0,0.32,-20.86,319.75,232.18,198.0,-0.08,0.37,0.32,XY24,NTT,MISC
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79872.0,-65288.0,141214.0,-0.50,-44.98,176.80,52.31,136.0,-0.46,0.58,6.91,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.42,93350.0,-35320.0,80617.0,0.04,-27.45,86.36,35.20,219.0,-0.44,0.68,7.14,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,49.0,X-SC,1.16,107542.0,5538.0,38382.0,1.61,5.43,35.69,43.06,122.0,0.14,0.78,23.47,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.31,108180.0,-19758.0,66888.0,-0.08,-15.44,61.83,36.84,143.0,-0.30,0.78,10.75,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.33,279180.0,-66776.0,133476.0,0.70,-19.30,47.81,19.28,1.0,-0.50,2.02,3.21,X40,ATH,IT
39,INFY,2275.00,-19.24,47.0,X-LC,7.64,314367.0,1479.0,170198.0,-0.94,0.47,54.14,54.87,3.0,0.01,2.28,7.15,X40,BTT,IT
41,ITC,452.00,-39.61,43.0,X-LC,2.83,195258.0,-4880.0,25774.0,0.63,-2.44,13.20,10.44,4.0,-0.19,1.41,2.99,X40,NTT,FMCG
82,VBL,671.64,-21.25,40.0,X-LC,10.67,283495.0,-32347.0,145008.0,0.62,-10.24,51.15,35.67,5.0,-0.22,2.05,2.19,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,46.0,H-LC,7.43,247800.0,-13819.0,133044.0,0.84,-5.28,53.69,45.57,7.0,-0.10,1.79,5.44,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.70,81005.0,-32544.0,72597.0,-0.14,-28.66,89.62,35.27,268.0,-0.45,0.59,98.26,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,40.0,H-SC,9.86,89835.0,-8145.0,29744.0,-0.05,-8.31,33.11,22.05,152.0,-0.27,0.65,30.58,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,106.53,58.0,H-SC,11.51,130140.0,-12339.0,31858.0,1.30,-8.66,24.48,13.70,163.0,-0.39,0.94,53.60,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,9.92,144585.0,-34486.0,100212.0,-0.57,-19.26,69.31,36.71,135.0,-0.34,1.05,43.46,XR,NTT,BANKS
33,ICICIGI,2260.25,-13.61,67.0,X-MC,1.72,148562.0,12429.0,18689.0,8.24,9.13,12.58,22.86,91.0,0.67,1.08,22.89,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.36,149328.0,108.0,144072.0,0.70,0.07,96.48,96.62,208.0,0.00,1.08,61.61,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,9.92,144585.0,-34486.0,100212.0,-0.57,-19.26,69.31,36.71,135.0,-0.34,1.05,43.46,XR,NTT,BANKS
31,HINDZINC,730.22,36.83,69.0,M-LC,8.91,222676.0,17600.0,94971.0,1.24,8.58,42.65,54.89,52.0,0.19,1.61,34.75,X5K,ATH,METALS
4,ANGELONE,3033.00,13.27,66.0,X-SC,9.88,200556.0,9550.0,48153.0,1.71,5.00,24.01,30.21,157.0,0.20,1.45,26.92,X40N,NTT,FINANCE
50,PGHH,17907.65,-25.92,63.0,X-MC,3.07,214485.0,13665.0,54136.0,0.37,6.80,25.24,33.76,80.0,0.25,1.55,12.36,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.54
1,25,44.07
2,50,75.95


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.39
LC    33.73
MC    21.93
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.65
X40      14.86
XY25     12.20
XR       11.00
X40N     10.11
AR        8.20
OX40N     7.66
X200      1.82
X5K       1.61
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.62
X-LC    22.58
X-MC    16.16
M-SC    12.68
M-LC     5.38
H-LC     4.74
X-SC     4.64
H-MC     3.84
M-MC     1.63
L-SC     1.45
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.12,-5.20,39.88
IT,13.01,-17.07,80.39
FINANCE,12.23,-11.88,62.59
MISC,6.91,-20.50,87.27
BANKS,6.43,-12.53,71.72
PAINTS,5.60,-19.46,37.27
ELECTRICAL,5.36,-11.09,50.07
INSURANCE,3.92,-2.39,40.67
AUTO,2.81,-47.82,118.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3344239.0,22
XR,1392769.0,14
AR,1196926.0,9
X40,788291.0,10
X40N,628715.0,8
OX40N,589460.0,10
XY25,497506.0,8
SR,266770.0,2
X5K,94971.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3467295.0,25
M-SC,1291023.0,15
X-LC,1119024.0,13
X-MC,1095580.0,11
X-SC,539743.0,6
H-MC,351108.0,3
H-LC,283717.0,3
M-LC,265190.0,4
L-SC,261036.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1205253.0      6
           XR         826048.0      7
           AR         822710.0      5
M-SC       XY24       699808.0      6
X-MC       XY24       589100.0      4
X-LC       X40        494406.0      5
           XY24       310053.0      2
X-SC       X40N       294637.0      4
H-SC       OX40N      275497.0      4
           SR         266770.0      2
X-LC       X40N       213906.0      3
X-MC       X40        213268.0      4
H-LC       AR         210067.0      2
H-MC       XY24       180642.0      1
X-MC       XY25       173040.0      2
L-SC       XR         170899.0      2
M-MC       XY24       167735.0      1
X-SC       XY24       164489.0      1
M-SC       AR         164149.0      2
           OX40N      151441.0      4
           XY25       144072.0      1
           XR         131553.0      2
X-MC       X40N       120172.0      1
M-LC       XR         105517.0      1
X-LC       XY25       100659.0      3
H-MC       XR          98081.0      1
M-LC       X5K         94971.0      1
L-SC       OX40N       90137.0      1
X-SC       X40         80617.0      1
H-LC       X200        73650.0      1
H-MC       OX40N       72385.0      1
H-SC       MH          71017.0      1
L-MC       XR          60671.0      1
L-LC       XY25        42192.0      1
M-LC       XY25        37543.0      1
           XY24        27159.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
